In [0]:
import tensorflow as tf
import sklearn 
import nltk
import numpy as np
import gensim
from gensim.models import KeyedVectors
import keras
from sklearn.model_selection import train_test_split

import pickle


from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
model = KeyedVectors.load_word2vec_format('/gdrive/My Drive/Pretrained_models/cc.en.300.vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
embeddings = {}
for key,val in model.vocab.items():
  embeddings[key] = model[key]  

In [0]:
print(len(embeddings))

2000000


In [0]:
import string
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
df = pd.read_csv('/gdrive/My Drive/amazon_only_english_normalised_book_reviews.csv')

In [0]:
main_lines = []
lines = df['normalised'].values.tolist()
for line in lines:
  tokens = word_tokenize(line)
  words = [word for word in tokens if word.isalpha()]
  main_lines.append(words)
  
  

In [0]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(main_lines)
sequences = tokenizer_obj.texts_to_sequences(main_lines)
word_index = tokenizer_obj.word_index
description = pad_sequences(sequences,maxlen = 150)
rating = df['rating'].values


In [0]:
print('shape of description tensor',description.shape)
print('shape of categories',rating.shape)

shape of description tensor (2993106, 150)
shape of categories (2993106,)


In [0]:
EMBEDDING_DIM = 300

In [0]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))


In [0]:
with open('/gdrive/My Drive/word_index.pkl','wb') as f1:
  pickle.dump(word_index,f1)

In [0]:
for word,i in word_index.items():
  if i > num_words:
    continue
  embedding_vector = embeddings.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    

In [0]:
nlp_model = Sequential()
embedding_layer = Embedding(num_words,EMBEDDING_DIM,embeddings_initializer = Constant(embedding_matrix),input_length = 150,trainable = False)
nlp_model.add(embedding_layer)
nlp_model.add(GRU(units=96,dropout=0.2,recurrent_dropout=0.2))
nlp_model.add(Dense(71,activation = 'softmax'))
nlp_model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
nlp_model.summary()

W0905 03:58:24.699965 139842046162816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0905 03:58:24.765012 139842046162816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 03:58:25.061614 139842046162816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0905 03:58:25.180266 139842046162816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0905 03:58:25.195819 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          13731300  
_________________________________________________________________
gru_1 (GRU)                  (None, 96)                114336    
_________________________________________________________________
dense_1 (Dense)              (None, 71)                6887      
Total params: 13,852,523
Trainable params: 121,223
Non-trainable params: 13,731,300
_________________________________________________________________


In [0]:
output_file = 'klooker_nlp_best_new'

In [0]:
checkpointer = ModelCheckpoint(filepath='/gdrive/My Drive/' + output_file + ".hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(description, categories, test_size=0.5, random_state=1)

In [0]:
labels_id = {i:index for index,i in enumerate(set(categories))}

In [0]:
import copy

tmp = np.zeros(71,dtype=np.int16)
y_train_sm = []
y_test_sm = []

for i in y_train:
  val = labels_id[i]
  new = copy.deepcopy(tmp)
  new[val] = 1
  y_train_sm.append(new)
for k in y_test:
  val1 = labels_id[k]
  new1 = copy.deepcopy(tmp)
  new1[val1] = 1
  y_test_sm.append(new1)
  

In [0]:
y_train_sm_arr = np.asarray(y_train_sm)
y_test_sm_arr = np.asarray(y_test_sm)

In [0]:
nlp_model.fit(x_train,y_train_sm_arr,batch_size=80,epochs = 150,validation_data=(x_test,y_test_sm_arr),verbose=2,callbacks=[checkpointer, earlystopper])